# Structured Output, 1-Use ollama.AsyncClient

In [23]:
import asyncio
import json
import ollama
from pydantic import BaseModel, ValidationError


#Only for Jupyter Notebooks
import nest_asyncio
nest_asyncio.apply()


class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]
            
async def main():          
         
    OLLAMA_MODEL = 'llama3.1:8b'      
    ollama_async_client = ollama.AsyncClient() 
    await ollama_async_client.pull(OLLAMA_MODEL)    
                         
    prompt = """
        Create a calendar event for a team meeting. The meeting is on 2024-09-01, is about 'Project Alpha Status', 
        and the participants are 'Alice', 'Bob', and 'Charlie'. Respond in JSON format. The JSON must contain the following keys exactly:
            - name: (a string for the event title)
            - date: (a string for the event date in YYYY-MM-DD format)
            - participants: (a list of strings for the names of participants)
            """
        
    response = await ollama_async_client.chat(        
                model=OLLAMA_MODEL, 
                messages=[{'role': 'user', 'content': prompt}],
                format='json',
                options={'temperature': 0.7},    # <-- Add the temperature parameter here                
                # ===> This does not work yet in ollama.py
                #output_model=CalendarEvent       
            )
          
    #print(response['message']['content'])
    try:            
            response_content = response['message']['content']
            data = json.loads(response_content)
            
            # ===> Validate and parse the JSON response into the CalendarEvent model
            event = CalendarEvent.model_validate(data)
            
            print(event.name)
            print(event.date)
            print(event.participants)
            
    except (json.JSONDecodeError, ValidationError) as e:
            print(f"Error parsing or validating JSON response: {e}")
            #print(f"Raw response content: {response_content}")

        
if __name__ == '__main__':
    # Run the main asynchronous function
    asyncio.run(main())

Project Alpha Status Meeting
2024-09-01
['Alice', 'Bob', 'Charlie']


# Use Instructor

In [ ]:

import asyncio
import json
import ollama
from pydantic import BaseModel, ValidationError
import instructor

#Only for Jupyter Notebooks
import nest_asyncio
nest_asyncio.apply()


class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]
            
            
async def main():    
    OLLAMA_MODEL = 'llama3.1:8b'      
    ollama_async_client = ollama.AsyncClient() 
    await ollama_async_client.pull(OLLAMA_MODEL)    
                         
    prompt = """
        Create a calendar event for a team meeting. The meeting is on 2024-09-01, is about 'Project Alpha Status', 
        and the participants are 'Alice', 'Bob', and 'Charlie'. Respond in JSON format. The JSON must contain the following keys exactly:
            - name: (a string for the event title)
            - date: (a string for the event date in YYYY-MM-DD format)
            - participants: (a list of strings for the names of participants)
            """
    client = instructor.from_provider("ollama/llama3")  # Ollama (local)
    
    print("type of client ===> ", type(client))
    
    response = client.chat.completions.create(        
            model=OLLAMA_MODEL, 
            messages=[{'role': 'user', 'content': prompt}],                
            temperature=0.7,    # <-- Add the temperature parameter here  
            response_model=CalendarEvent       
    )
    
    try:
        print(response)
                        
    except (json.JSONDecodeError, ValidationError) as e:
            print(f"Error parsing or validating JSON response: {e}")
            #print(f"Raw response content: {response_content}")
        
if __name__ == '__main__':
    # Run the main asynchronous function
    asyncio.run(main())

type of client ===>  <class 'instructor.core.client.Instructor'>
name='Project Alpha Status' date='2024-09-01' participants=['Alice', 'Bob', 'Charlie']


# Output, 2-Use OpenAI 

In [14]:
import asyncio
import instructor
from openai import OpenAI
from pydantic import BaseModel

#Only for Jupyter Notebooks
import nest_asyncio
nest_asyncio.apply()

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]
    
            
async def main():     
    # Connect OpenAI client to local Ollama server
    client = OpenAI(
        base_url="http://localhost:11434/v1/",
        api_key="not-needed"   # Ollama ignores it
    )

    # Add instructor wrapper for Pydantic validation
    #client = instructor.patch(client)
    
    resp = client.chat.completions.create(
        model="llama3.1:8b",
        messages=[
            {"role": "system", "content": "Extract the event information."},
            {"role": "user", "content": "Create a calendar event for 2024-09-01 about Project Alpha with Alice, Bob, Charlie"}
            ],
        temperature=0.3,
        #response_model=CalendarEvent,  
        #strict = True                 
    )

    #
    #print(resp.model_dump_json(indent=2))
    #print(resp.name)
    #print(resp.date)
    #print(resp.participants)
    
    #
    #print(type(resp))  # <class '__main__.CalendarEvent'>
    print(resp)  

        
if __name__ == '__main__':
    # Run the main asynchronous function
    asyncio.run(main())

ChatCompletion(id='chatcmpl-845', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Here is a potential calendar event:\n\n**Event Title:** Project Alpha Meeting\n**Date and Time:** September 1, 2024 (all day)\n**Location:** [Insert location or virtual meeting details]\n**Attendees:**\n\t* Alice\n\t* Bob\n\t* Charlie\n**Description:** Project Alpha meeting to discuss progress and next steps.\n\nAlternatively, if you want a more detailed event:\n\n**Event Title:** Project Alpha Meeting\n**Date and Time:** September 1, 2024 (10:00 AM - 12:00 PM)\n**Location:** Conference Room A, [Insert location]\n**Attendees:**\n\t* Alice\n\t* Bob\n\t* Charlie\n**Description:** Project Alpha meeting to discuss progress and next steps. Agenda items include:\n\t+ Review of current project status\n\t+ Discussion of upcoming milestones\n\t+ Brainstorming for potential solutions to challenges', refusal=None, role='assistant', annotations=None, audio=None, fu

# 3. Structured Output(Ollama), 2-Use OpenAI and instructor 

In [ ]:
import asyncio
import instructor
from openai import OpenAI
from pydantic import BaseModel

#Only for Jupyter Notebooks
import nest_asyncio
nest_asyncio.apply()

#--------------------------------------
# pydantic BaseModel
#--------------------------------------
class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]
    
            
async def main():     
    # Connect OpenAI client to local Ollama server
    client = OpenAI(
        base_url="http://localhost:11434/v1/",
        api_key="not-needed"   # Ollama ignores it
    )

    #  instructor, patch 
    client = instructor.patch(client)
    
    resp = client.chat.completions.create(
        model="llama3.1:8b",
        messages=[
            {"role": "system", "content": "Extract the event information."},
            {"role": "user", "content": "Create a calendar event for 2024-09-01 about Project Alpha with Alice, Bob, Charlie"}
            ],
        temperature=0.3,
        # Instructor and Patch
        response_model=CalendarEvent,  
        strict = True,                 
    )

    #
    print(resp.model_dump_json(indent=2))
    print(resp.name)
    print(resp.date)
    print(resp.participants)
    
    #
    print(type(resp))  # <class '__main__.CalendarEvent'>
    print(resp)  

        
if __name__ == '__main__':
    # Run the main asynchronous function
    asyncio.run(main())

{
  "name": "Project Alpha",
  "date": "2024-09-01",
  "participants": [
    "Alice",
    "Bob",
    "Charlie"
  ]
}
Project Alpha
2024-09-01
['Alice', 'Bob', 'Charlie']
<class '__main__.CalendarEvent'>
name='Project Alpha' date='2024-09-01' participants=['Alice', 'Bob', 'Charlie']
